# Part 1: Input and Dataset Construction

This notebook covers the complete workflow for preparing InSAR data for the Damage Proxy Mapping (DPM) analysis.

---

## 1. Data Preprocessing (External)

Before running this notebook, the **Sentinel-1 SAR data** must be preprocessed using `stackSentinel.py` (https://github.com/isce-framework/isce2/blob/main/contrib/stack/topsStack/stackSentinel.py), which automates:

- **SAR Image Collection** - Multiple Sentinel-1 SAR images for the desired time period
- **Baseline Estimation** - Determining geometric relationships between image pairs
- **Multi-looking** - Reducing speckle noise
- **Image Registration (Coregistration)** - Aligning images for interferometric analysis
- **Filtering** - Noise reduction and phase filtering
- **Interferogram Generation** - Creating interferograms from adjacent image pairs
- **Coherence Map Generation** - Output: `filt_fine.cor` (filtered coherence files)

## 2. This Notebook Workflow

1. **SAR Preprocessing Overview** - Understanding the `stackSentinel.py` output
2. **InSAR Data Processing Utilities** - Core functions for reading/writing ISCE2 files and coordinate transformations
3. **Batch Cropping** - Extracting Region of Interest (ROI) from interferogram coherence files
4. **Dataset Generation** - Building time series data structure and calculating phase standard deviation

---

## Input Files

| File | Description | Source |
|------|-------------|--------|
| `filt_fine.cor` | Filtered coherence maps | `stackSentinel.py` output (final step after coregistration) |
| `lat.rdr` | Latitude lookup table | `stackSentinel.py` geometry output |
| `lon.rdr` | Longitude lookup table | `stackSentinel.py` geometry output |

## Output Files

| File | Description | Usage |
|------|-------------|-------|
| `data.npy` | Pre-event coherence time series (H, W, T) | LSTM training input |
| `data_std.npy` | Phase standard deviation time series | LSTM training input |
| `geninue.npy` | Co-event coherence data | Damage score calculation |
| `geninue_std.npy` | Co-event phase standard deviation | Damage score calculation |
| `dates.pkl` | Interferogram date pairs | Temporal feature extraction |
| `lat_cropped.rdr` | Cropped latitude lookup table | Geocoding |
| `lon_cropped.rdr` | Cropped longitude lookup table | Geocoding |

---

## stackSentinel.py Overview

`stackSentinel.py` is a key script in the ISCE processing chain, designed to automate the preprocessing of Sentinel-1 SAR data. It orchestrates various steps from initial data unpacking to generating interferograms, coherence maps, and applying necessary filters. Below is an overview of its workflow:

### 1. Data Input and Unpacking
The script begins by organizing and unpacking Sentinel-1 SAR data. It sets up directories and prepares the raw SAR data for further processing. This typically involves downloading and unpacking the raw Sentinel-1 products.

### 2. Interferogram Calculation
After unpacking, `stackSentinel.py` facilitates the calculation of interferograms between pairs of SAR images. These interferograms are the basis for detecting ground displacement.

### 3. Coherence Map Generation
The script generates coherence maps that quantify the correlation between SAR image pairs. High coherence values indicate stable surfaces, while low coherence values suggest changes or noisy areas.

### 4. Phase Unwrapping and Topographic Phase Removal
To process the interferograms, the script performs phase unwrapping, eliminating ambiguities in phase values. It also removes the topographic phase component, isolating the deformation signal from the topographic effects.

### 5. Filtering and Refining
Several filtering steps are included in the script to remove noise from the interferograms and coherence maps. This step enhances the quality of the data by smoothing and reducing phase errors.

### 6. Coherence Thresholding and Final Filtering
The script applies a coherence threshold, removing low-coherence regions, and further refines the coherence map by filtering out any remaining noise.

### 7. Output: `filt_fine.cor`
The final output of the script is a filtered and processed coherence map, stored in the `/merged/interferograms/` directory. This file, `filt_fine.cor`, serves as the foundation for any subsequent analysis of surface displacements or deformations.

### Purpose
The main purpose of `stackSentinel.py` is to automate the preprocessing steps for Sentinel-1 data, ensuring that the generated interferograms and coherence maps are of high quality and suitable for further analysis.




---

# InSAR Data Processing Utilities

This script provides several utilities to work with InSAR data, specifically focusing on geospatial transformations, file reading and writing, and data processing. It contains functions for converting geographic bounding boxes into SAR row/column indices, reading ISCE2 formatted files, and writing output data back to various file formats, including ISCE2 and GDAL. These utilities are particularly useful for processing and manipulating interferogram data from InSAR missions.

## Functions

### 1. `bbox2SAR(lat_min, lat_max, lon_min, lon_max, lat_data, lon_data)`

Converts a geographic bounding box (latitude and longitude) into SAR (Synthetic Aperture Radar) row and column indices in the azimuth and range directions.

* **Arguments**:

  * `lat_min`: Minimum latitude.
  * `lat_max`: Maximum latitude.
  * `lon_min`: Minimum longitude.
  * `lon_max`: Maximum longitude.
  * `lat_data`: A 2D array of latitude values for the SAR grid.
  * `lon_data`: A 2D array of longitude values for the SAR grid.
* **Returns**:

  * A list containing the SAR region in terms of row and column indices that correspond to the geographic bounding box.

### 2. `read_isce_file(file)`

Reads an ISCE2 formatted file (commonly used in InSAR data) and returns it as a NumPy array.

* **Arguments**:

  * `file`: Path to the ISCE2 formatted file (e.g., `.unw` file).
* **Returns**:

  * A NumPy array representing the data in the ISCE2 file.

### 3. `write_gdal_file(arr, output_filepath, data_type=gdal.GDT_Float32)`

Writes a NumPy array to a file using GDAL, specifically in the ENVI format. This function handles both 2D and 3D arrays.

* **Arguments**:

  * `arr`: The NumPy array to write to the file.
  * `output_filepath`: The path to the output file.
  * `data_type`: The data type for the output file (default is `gdal.GDT_Float32`).
* **Returns**:

  * None. The function writes the array to the specified file.

### 4. `write_arr2file(arr, output_filepath)`

Writes a NumPy array to an ISCE2 formatted file (such as `.unw`, `.cor`, or `.rdr`).

* **Arguments**:

  * `arr`: The NumPy array to convert and write.
  * `output_filepath`: The path to the output ISCE2 file.
* **Returns**:

  * None. The function writes the array to the specified ISCE2 file.


In [1]:
   
from mintpy.utils.writefile import * 
from osgeo import gdal  
import os, sys
import numpy as np
import math
import geopandas as gpd
from shapely.geometry import Polygon


## geobbox to SAR row/col in the azimuth&range direction 
def bbox2SAR(lat_min, lat_max, lon_min, lon_max, lat_data, lon_data):
    #  convert geographic bounding box to SAR ROI indicies. (azimuth and range)
    # <1> lat_min (float)     : minimum latitude
    # <2> lat_max (float)     : maximum latitude
    # <3> lon_min (float)     : minimum longitude
    # <4> lon_max (float)     : maximum longitude
    # <5> lat_data (np.array) : the latitude lookuptable lat.rdr  
    # <6> lon_data (np.array) : the lontitude lookuptable lon.rdr
    # <return> region_rec: return the SAR row col list 
    S, N, W, E = lat_min, lat_max, lon_min, lon_max
    geo_coord = [W, N, E, S]
    data_map =  (lon_data >= geo_coord[0])*(lon_data <= geo_coord[2])*(lat_data>=geo_coord[3])*(lat_data<=geo_coord[1])
    region_list = np.argwhere(data_map==1)
    region_rec = [10*math.floor(region_list[:,0].min()/10),10*math.ceil(region_list[:,0].max()/10),\
              10*math.floor(region_list[:,1].min()/10),10*math.ceil(region_list[:,1].max()/10)]
    return region_rec
    
## convert ISCE2 formatted file to npArray
def read_isce_file(file):
    # convert a GDAL_realiable file (usually ISCE2 file) to a numpy array 
    # <1> file(str): a string-path of input file 
    _, ext = os.path.splitext(file)
    ds = gdal.Open(file,gdal.GA_ReadOnly)
    print("input dataset BandsCount : ", ds.RasterCount)
    ## get the phase band for the unw data
    if ext != ".unw":band = ds.GetRasterBand(1)
    else:band = band = ds.GetRasterBand(2)
    data = np.expand_dims(band.ReadAsArray(), 2)
    W, L, N = data.shape
    loader = np.zeros([W, L, N], dtype=np.float32)
    loader[:,:,0] = data[:,:,0]
    return loader


## there may be some problems in the funtion @
## don`t use it 
def write_gdal_file(arr, output_filepath, data_type=gdal.GDT_Float32):
    if len(arr.shape) == 2:
        rows, cols = arr.shape
        bands = 1
        data_to_write = arr
    elif len(arr.shape) == 3:
        rows, cols, bands = arr.shape
        data_to_write = arr[:, :, 0] if bands == 1 else arr
    else:
        raise ValueError("Array must be 2D or 3D")
    driver = gdal.GetDriverByName('ENVI')    
    dataset = driver.Create(output_filepath, cols, rows, 1, data_type)
    if dataset is None:
        raise RuntimeError(f"Could not create file: {output_filepath}")
    if len(arr.shape) == 2:
        dataset.GetRasterBand(1).WriteArray(data_to_write)
    else:
        dataset.GetRasterBand(1).WriteArray(data_to_write[:, :])
    dataset.FlushCache()
    dataset = None
    print(f"GDAL write {output_filepath} finished")


## write np.array to the ISCE2 file according to the input array
def write_arr2file(arr, output_filepath):
    ## write np.array to the ISCE2 file according to the input array
    ## <1> arr(numpy.array) : the numpy array to convert
    ## <2> out_path (str)   : the output path of the arr2ISCEfile
    mirror_dic = {
        ".unw": "MOD_isce_unw", 
        ".cor": "isce_cor", 
        ".rdr": "isce_cor", 
        ".int": "isce_int",
        ".full": "envi"
    }
    _, ext = os.path.splitext(output_filepath)
    if ext == ".full":
        write_gdal_file(arr, output_filepath)
        return
    if ext not in mirror_dic:
        raise ValueError(f"Unsupported file extension: {ext}")
    file_type = mirror_dic[ext]
    write_isce_file(
        data = arr[:,:,0],
        out_file = output_filepath ,
        file_type = file_type
    )
    print(f"write {output_filepath} finished ")

# Batch Crop of `filt_fine.cor` Files for InSAR Data

This script provides a utility to batch crop `filt_fine.cor` files along with the corresponding latitude (`lat.rdr`) and longitude (`lon.rdr`) files from an InSAR dataset. The cropping is done based on a specified geographic region (latitude and longitude bounds), which is then converted into SAR image coordinates. The output files, which include the cropped versions of the `filt_fine.cor`, latitude, and longitude files, are saved to the designated output directory.

## Key Functions

1. **`batch_crop_filt_fine_cor()`**:

   * This function processes all `filt_fine.cor` files in a given directory, crops them according to the specified geographic bounds, and saves the cropped files.
   * It also crops the corresponding latitude and longitude files from the geometry reference path.

2. **`crop_single_band_file()`**:

   * Crops a single band file (such as a `filt_fine.cor` file) based on the provided coordinates and saves the cropped data.
   * It checks whether the cropping region exceeds the data boundaries and adjusts accordingly.

3. **`read_isce_file()`** and **`write_arr2file()`**:

   * These helper functions are used for reading ISCE-formatted files and writing arrays to the corresponding output files (e.g., `.rdr`, `.cor`).

## Script Overview

* **Input**:

  * A directory containing `filt_fine.cor` files (`base_path`).
  * A reference geometry path containing `lat.rdr` and `lon.rdr` files (`geom_reference_path`).
* **Output**:

  * Cropped versions of the `filt_fine.cor`, latitude (`lat.rdr`), and longitude (`lon.rdr`) files saved to a specified output directory (`output_base_path`).

### Geographic Region for Cropping

The script is set to crop the data based on a fixed geographic bounding box:

* **Latitude**: [42.625, 42.635]
* **Longitude**: [13.28, 13.30]

These bounds are used to define the region to be extracted from the InSAR dataset.

## Example Workflow

1. **Input Data**:

   * `filt_fine.cor` files for interferogram data.
   * `lat.rdr` and `lon.rdr` files for the corresponding latitude and longitude grids.

2. **Cropping**:

   * The latitude and longitude data are used to find the SAR region corresponding to the geographic bounding box.
   * Each `filt_fine.cor` file is then cropped based on this region.

3. **Output**:

   * The cropped files are saved in the specified output directory.



In [2]:
import os
import numpy as np
from osgeo import gdal
import traceback
import re


def batch_crop_filt_fine_cor(base_path, geom_reference_path, output_base_path):
    """
    Batch crop all filt_fine.cor files and simultaneously crop corresponding lat and lon files.
    
    Parameters:
    base_path: The base path containing filt_fine.cor files
    geom_reference_path: The geometry reference path containing lat and lon files
    output_base_path: The base path where cropped files will be saved
    """
    
    # Use the provided latitude and longitude range
    lat_min, lat_max = 42.625, 42.635
    lon_min, lon_max = 13.28, 13.30
    
    print(f"Using cropping region: Latitude[{lat_min}, {lat_max}], Longitude[{lon_min}, {lon_max}]")
    
    # Look for lat and lon files
    lat_file = os.path.join(geom_reference_path, "lat.rdr")
    lon_file = os.path.join(geom_reference_path, "lon.rdr")
    
    if not os.path.exists(lat_file):
        # Try other possible file names
        lat_files = [f for f in os.listdir(geom_reference_path) if "lat" in f.lower() and not f.startswith('.')]
        if lat_files:
            lat_file = os.path.join(geom_reference_path, lat_files[0])
            print(f"Using found latitude file: {lat_file}")
        else:
            print(f"Error: Could not find lat file in {geom_reference_path}")
            return
    
    if not os.path.exists(lon_file):
        # Try other possible file names
        lon_files = [f for f in os.listdir(geom_reference_path) if "lon" in f.lower() and not f.startswith('.')]
        if lon_files:
            lon_file = os.path.join(geom_reference_path, lon_files[0])
            print(f"Using found longitude file: {lon_file}")
        else:
            print(f"Error: Could not find lon file in {geom_reference_path}")
            return
    
    print(f"Using lat file: {lat_file}")
    print(f"Using lon file: {lon_file}")
    
    # Read lat and lon data
    lat_data = read_isce_file(lat_file)
    lon_data = read_isce_file(lon_file)
    print(f"Lat file shape: {lat_data.shape}")
    print(f"Lon file shape: {lon_data.shape}")
    
    # Calculate the cropping region (SAR coordinates)
    region_rec = bbox2SAR(lat_min, lat_max, lon_min, lon_max, lat_data, lon_data)
    if len(region_rec) != 4:
        print("Invalid region coordinates")
        return
    
    y_min, y_max, x_min, x_max = region_rec
    height = y_max - y_min
    width = x_max - x_min
    print(f"Geographical coordinates converted to SAR coordinates: [{y_min}, {y_max}, {x_min}, {x_max}]")
    print(f"SAR image range: Rows {y_min} to {y_max}, Columns {x_min} to {x_max}")
    print(f"SAR image size: {height} × {width} (Height × Width)")
    
    # Check if the cropping region is valid
    if height <= 0 or width <= 0:
        print("Error: Invalid cropping region, height or width is 0")
        return
    
    # Crop the lat and lon files
    lat_cropped = lat_data[y_min:y_max, x_min:x_max]
    lon_cropped = lon_data[y_min:y_max, x_min:x_max]
    
    # Save the cropped lat and lon files
    lat_output = os.path.join(output_base_path, "lat_cropped.rdr")
    lon_output = os.path.join(output_base_path, "lon_cropped.rdr")
    
    write_arr2file(lat_cropped, lat_output)
    write_arr2file(lon_cropped, lon_output)
    
    print(f"Cropped lat file saved: {lat_output}, shape: {lat_cropped.shape}")
    print(f"Cropped lon file saved: {lon_output}, shape: {lon_cropped.shape}")
    
    # Walk through the directory structure and find filt_fine.cor files
    cor_files = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == "filt_fine.cor":
                cor_files.append(os.path.join(root, file))
    
    print(f"Found {len(cor_files)} filt_fine.cor files")
    
    # Process each file
    for i, cor_file in enumerate(cor_files):
        print(f"\nProcessing file {i+1}/{len(cor_files)}: {cor_file}")
        
        try:
            # Extract date information from file path
            # Assuming the path format is: .../YYYYMMDD_YYYYMMDD/filt_fine.cor
            date_match = re.search(r'(\d{8}_\d{8})', cor_file)
            if date_match:
                date_str = date_match.group(1)
                print(f"Extracted date information: {date_str}")
            else:
                # If no date is found, use the parent directory name
                parent_dir = os.path.basename(os.path.dirname(cor_file))
                date_str = parent_dir
                print(f"Using parent directory name as date: {date_str}")
            
            # Create the output file name
            output_file = os.path.join(output_base_path, f"{date_str}_filt_fine.cor")
            
            # Check if the output file already exists
            if os.path.exists(output_file):
                print(f"Skipping existing file: {output_file}")
                continue
            
            # Call the cropping function
            crop_single_band_file(
                cor_file, y_min, y_max, x_min, x_max, output_file
            )
            
            print(f"Successfully cropped: {output_file}")
            
        except Exception as e:
            print(f"Error processing file {cor_file}: {str(e)}")
            traceback.print_exc()

def crop_single_band_file(file, y_min, y_max, x_min, x_max, out_file):
    """Crop a single band file (such as filt_fine.cor)"""
    
    # Read data
    data = read_isce_file(file)
    print(f"Original data shape: {data.shape}")
    
    # Check if the cropping region exceeds data boundaries
    data_height, data_width = data.shape[0], data.shape[1]
    if y_max > data_height or x_max > data_width:
        print(f"Warning: Cropping region exceeds data boundaries, adjusting cropping region")
        y_max = min(y_max, data_height)
        x_max = min(x_max, data_width)
        print(f"Adjusted SAR coordinates: [{y_min}, {y_max}, {x_min}, {x_max}]")
    
    # Crop data
    if len(data.shape) == 2:  # Single band data
        data_crop = data[y_min:y_max, x_min:x_max]
    elif len(data.shape) == 3:  # Multi-band data
        data_crop = data[y_min:y_max, x_min:x_max, :]
    else:
        print(f"Unsupported data dimensions: {len(data.shape)}")
        return
    
    print(f"Cropped data shape: {data_crop.shape}")
    
    # Calculate data statistics
    if np.iscomplexobj(data_crop):
        print(f"Complex data range: Min magnitude={np.min(np.abs(data_crop)):.4f}, Max magnitude={np.max(np.abs(data_crop)):.4f}")
    else:
        print(f"Data range: Min={np.min(data_crop):.4f}, Max={np.max(data_crop):.4f}")
    
    # Save cropped data
    if not file.endswith("full"):
        write_arr2file(data_crop, out_file)
    else:
        write_gdal_file(data_crop, out_file)

def main():
    """Main function"""
    
    # Set path parameters
    base_path = "/data6/WORKDIR/AmatriceSenDT22/merged/interferograms"
    geom_reference_path = "/data6/WORKDIR/AmatriceSenDT22/merged/geom_reference"
    output_base_path = "/data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped"  # Output directory
    
    # Ensure the output directory exists
    os.makedirs(output_base_path, exist_ok=True)
    
    print("Starting batch crop of filt_fine.cor files")
    print(f"Input path: {base_path}")
    print(f"Geometry reference path: {geom_reference_path}")
    print(f"Output path: {output_base_path}")
    
    # Execute batch cropping
    batch_crop_filt_fine_cor(
        base_path, geom_reference_path, output_base_path
    )
    
    print("\nBatch cropping complete!")

if __name__ == '__main__':
    main()


Starting batch crop of filt_fine.cor files
Input path: /data6/WORKDIR/AmatriceSenDT22/merged/interferograms
Geometry reference path: /data6/WORKDIR/AmatriceSenDT22/merged/geom_reference
Output path: /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped
Using cropping region: Latitude[42.625, 42.635], Longitude[13.28, 13.3]
Using lat file: /data6/WORKDIR/AmatriceSenDT22/merged/geom_reference/lat.rdr
Using lon file: /data6/WORKDIR/AmatriceSenDT22/merged/geom_reference/lon.rdr
input dataset BandsCount :  1
input dataset BandsCount :  1
Lat file shape: (2858, 26031, 1)
Lon file shape: (2858, 26031, 1)
Geographical coordinates converted to SAR coordinates: [950, 1060, 21330, 21870]
SAR image range: Rows 950 to 1060, Columns 21330 to 21870
SAR image size: 110 × 540 (Height × Width)
write file: /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped/lat_cropped.rdr
write file: /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped/lat_cropped.rdr.xml
write file: /data6/WORKD

# InSAR Dataset Processing for Pre- and Post-Earthquake Data

This script processes a batch of InSAR coherence files (`filt_fine.cor`), extracting both pre- and post-earthquake data to generate an InSAR dataset for further analysis. The main steps involve reading the coherence files, creating an InSAR timeseries (which contains pre-earthquake data), generating post-earthquake data, and saving the dataset along with calculated statistics (such as standard deviation). The result is a collection of processed data saved in `.npy` format, which can be used for machine learning, anomaly detection, or further scientific analysis.

## Key Features

1. **File Handling**:

   * The script finds all coherence files (`filt_fine.cor`) in a specified directory, sorts them by date, and processes them in chronological order.

2. **Data Extraction**:

   * Builds an InSAR timeseries using the selected coherence files before the earthquake event (specifically before 2016-08-24).
   * Extracts the amplitude, phase, or real part of the data based on user choice.
   * Generates post-earthquake data (`geninue.npy`) by using the last available coherence file after the earthquake event.

3. **Standard Deviation Calculation**:

   * Computes the standard deviation for each coherence band in the InSAR timeseries. This helps assess the coherence quality in each pixel.

4. **Saving Processed Data**:

   * Saves the processed InSAR timeseries (`data.npy`), corresponding dates (`dates.pkl`), post-earthquake data (`geninue.npy`), and standard deviation files (`data_std.npy` and `geninue_std.npy`).

5. **Output Directory Structure**:

   * The processed dataset is saved in a specified subfolder of the output directory for easy access.

## Workflow Overview

1. **Input**:

   * Coherence files (`filt_fine.cor`) containing interferometric phase data.
   * The Earthquake event date, used to divide the dataset into pre- and post-event data.

2. **Processing**:

   * The script reads and processes the coherence files, constructs an InSAR timeseries with pre-earthquake data, and calculates standard deviations for the data.
   * After the earthquake event, the script extracts the post-earthquake coherence data (post-event `geninue.npy`).

3. **Output**:

   * The final output consists of:

     * **InSAR timeseries** (pre-earthquake data).
     * **Post-earthquake data** (`geninue.npy`).
     * **Standard deviation** for both the pre- and post-earthquake data.

### Example Directory Structure

```
/data6/WORKDIR/AmatriceSenDT22/
├── merged/
│   ├── interferograms/
│   │   ├── cropped/   # Contains cropped coherence files
│   ├── interferograms/cropped/dataset/   # Output folder for saved dataset
│   ├── geom_reference/  # Geometry reference files (lat, lon, etc.)
```

In [3]:
import os
import datetime
import numpy as np
import pickle
import shutil
import re
from osgeo import gdal

# Set directory paths
cropped_dir = "/data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped"
output_dir = "/data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped"
next_date = "20160821_20160902"  # Predicted time period

# Earthquake event date
event_date = datetime.datetime(2016, 8, 24)

def find_cor_files_sorted(cropped_dir):
    """
    Find all *_filt_fine.cor files under cropped_dir,
    sort by the start date, and return [(start_datetime, date_str, full_path), ...]
    where date_str is in the format '20160716_20160728'.
    """
    file_infos = []
    for root, dirs, files in os.walk(cropped_dir):
        for fname in files:
            if not fname.endswith("filt_fine.cor"):
                continue
            m = re.search(r"(\d{8}_\d{8})", fname)
            if not m:
                print(f"Skipping file without date pattern: {os.path.join(root, fname)}")
                continue
            date_str = m.group(1)
            start_str, end_str = date_str.split("_")
            start_dt = datetime.datetime.strptime(start_str, "%Y%m%d")
            full_path = os.path.join(root, fname)
            file_infos.append((start_dt, date_str, full_path))

    # Sort by start date
    file_infos.sort(key=lambda x: x[0])
    print(f"Found {len(file_infos)} cropped .cor files")
    for dt, dstr, path in file_infos:
        print(dt.date(), dstr, "->", path)
    return file_infos

def read_isce_file(file):
    """
    Read ISCE2 formatted files (such as .unw files) and return as a NumPy array.
    """
    _, ext = os.path.splitext(file)
    ds = gdal.Open(file, gdal.GA_ReadOnly)
    print("Input dataset BandsCount:", ds.RasterCount)

    # If it's a .unw file, get the phase band data
    if ext != ".unw":
        band = ds.GetRasterBand(1)
    else:
        band = ds.GetRasterBand(2)
        
    data = np.expand_dims(band.ReadAsArray(), 2)
    print(f"Data shape: {data.shape}")
    W, L, N = data.shape
    loader = np.zeros([W, L, N], dtype=np.float32)
    loader[:, :, 0] = data[:, :, 0]
    return loader

def build_insar_timeseries_from_cor(file_infos, use="amplitude"):
    """
    Build InSAR timeseries (H, W, T-1) from sorted .cor files
    
    use:
      - "amplitude": Use |cor|
      - "phase": Use np.angle(cor)
      - "real": Use np.real(cor)
    """
    if len(file_infos) == 0:
        raise RuntimeError("No .cor files found")

    # Read the first file to determine size
    _, first_datestr, first_path = file_infos[0]
    first_data = read_isce_file(first_path)  # Should return 2D array (H, W), possibly complex
    print(f"First image {first_datestr} shape: {first_data.shape}, dtype: {first_data.dtype}")

    # If first_data is a 3D array (H, W, 1), use [:2] to get H and W
    H, W = first_data.shape[:2]
    T = len(file_infos)

    # The last dimension will be T-1, as the data before the earthquake
    timeseries = np.zeros((H, W, T-1), dtype=np.float32)
    dates = []

    # Use only the first T-1 files, the last one is for geninue.npy
    for t, (start_dt, date_str, path) in enumerate(file_infos[:-1]):  # Use only the first T-1 files
        print(f"[{t+1}/{T-1}] Reading {date_str} -> {path}")
        arr = read_isce_file(path)

        if arr.shape[:2] != (H, W):
            raise ValueError(f"File {path} shape {arr.shape} does not match the first image {first_data.shape}")

        if np.iscomplexobj(arr):
            if use == "amplitude":
                arr_use = np.abs(arr)
            elif use == "phase":
                arr_use = np.angle(arr)
            elif use == "real":
                arr_use = np.real(arr)
            else:
                raise ValueError(f"Unknown 'use' option: {use}")
        else:
            arr_use = arr

        # Convert to a 2D array (H, W), and store it in the timeseries
        timeseries[:, :, t] = arr_use[:, :, 0].astype(np.float32)
        dates.append(date_str)

    print(f"Final timeseries shape: {timeseries.shape} (H, W, T-1)")
    return timeseries, dates

def calculate_std_from_cor(cor, chunk_size=50):
    """
    Calculate standard deviation from coherence image
    Calculate standard deviation for each band
    """
    rows, cols, bands = cor.shape
    result = np.full_like(cor, np.nan, dtype=np.float32)
    
    # Add numerical stability factor
    epsilon = 1e-8
    
    # Iterate over the whole image in chunks (no overlap)
    for i in range(0, rows, chunk_size):
        for j in range(0, cols, chunk_size):
            # Calculate the boundaries for the current chunk
            end_i = min(i + chunk_size, rows)
            end_j = min(j + chunk_size, cols)
            
            # Extract current chunk data
            chunk = cor[i:end_i, j:end_j, :]  # Process all bands
            
            # Calculate the standard deviation for each band separately
            for band in range(bands):
                band_chunk = chunk[:, :, band]
                with np.errstate(divide='ignore', invalid='ignore'):
                    # Calculate standard deviation: std = sqrt((1 - cor^2) / 2*cor^2)
                    denominator = band_chunk**2
                    valid_mask = (denominator > epsilon)
                    std_chunk = np.where(valid_mask,
                                         np.sqrt((1 - denominator) / (2*denominator)),
                                         0.0)
                
                # Store the standard deviation in the corresponding position
                result[i:end_i, j:end_j, band] = std_chunk
    
    # Retain original invalid regions
    nan_mask = np.isnan(cor)
    zero_mask = (cor == 0)
    result[nan_mask] = np.nan
    result[zero_mask] = 0.0
    
    return result

def save_dataset(timeseries, dates, output_subfolder, geninue_data=None):
    """
    Save dataset (.npy and .pkl) to specified subfolder
    """
    os.makedirs(output_subfolder, exist_ok=True)
    
    # Save data
    data_path = os.path.join(output_subfolder, "data.npy")
    np.save(data_path, timeseries)
    print(f"Dataset saved to {data_path}")

    # Save dates
    dates_path = os.path.join(output_subfolder, "dates.pkl")
    with open(dates_path, "wb") as f:
        pickle.dump(dates, f)
    print(f"Dates list saved to {dates_path}")

    # Save geninue.npy (post-earthquake cor)
    if geninue_data is not None:
        geninue_path = os.path.join(output_subfolder, "geninue.npy")
        np.save(geninue_path, geninue_data)
        print(f"Saved geninue.npy to {geninue_path}")

    # Save standard deviation data
    data_std = calculate_std_from_cor(timeseries)
    data_std_path = os.path.join(output_subfolder, "data_std.npy")
    np.save(data_std_path, data_std)
    print(f"Saved standard deviation data to {data_std_path}")

    if geninue_data is not None:
        # Ensure geninue_data is a 3D array
        if geninue_data.ndim == 2:
            geninue_data = np.expand_dims(geninue_data, axis=-1)  # Convert to a 3D array, shape (rows, cols, 1)
        
        geninue_std = calculate_std_from_cor(geninue_data)
        geninue_std_path = os.path.join(output_subfolder, "geninue_std.npy")
        np.save(geninue_std_path, geninue_std)
        print(f"Saved geninue_std.npy to {geninue_std_path}")

def main():
    # Get cropped files
    file_infos = find_cor_files_sorted(cropped_dir)

    # Select training set (before 20160824)
    train_file_infos = [info for info in file_infos if info[0] < event_date]

    # Build training set data
    timeseries, dates = build_insar_timeseries_from_cor(train_file_infos, use="amplitude")

    # Generate geninue.npy (post-earthquake cor data, i.e., the last time interference pair's cor)
    _, geninue_date_str, geninue_path = file_infos[-1]  # Last time period data
    geninue_data = read_isce_file(geninue_path)
    geninue_data = np.abs(geninue_data[:, :, 0])  # Get amplitude and remove the last dimension

    # Save dataset
    output_subfolder = os.path.join(output_dir, "dataset")
    save_dataset(timeseries, dates, output_subfolder, geninue_data)

if __name__ == "__main__":
    main()


Found 10 cropped .cor files
2016-03-06 20160306_20160330 -> /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped/20160306_20160330_filt_fine.cor
2016-03-30 20160330_20160517 -> /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped/20160330_20160517_filt_fine.cor
2016-05-17 20160517_20160529 -> /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped/20160517_20160529_filt_fine.cor
2016-05-29 20160529_20160610 -> /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped/20160529_20160610_filt_fine.cor
2016-06-10 20160610_20160704 -> /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped/20160610_20160704_filt_fine.cor
2016-07-04 20160704_20160716 -> /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped/20160704_20160716_filt_fine.cor
2016-07-16 20160716_20160728 -> /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped/20160716_20160728_filt_fine.cor
2016-07-28 20160728_20160809 -> /data6/WORKDIR/AmatriceSenDT22/merged/interferograms/cropped/2016